In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
%load_ext cudf.pandas
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split, KFold
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import explained_variance_score, mean_absolute_error, mean_squared_error, r2_score
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor, early_stopping, log_evaluation
from catboost import CatBoostRegressor

from sklearn.linear_model import Ridge, Lasso, BayesianRidge

%matplotlib inline
plt.style.use('seaborn-v0_8')
plt.rc('figure', figsize=(10,6), dpi=180)
plt.rc('axes', labelweight='bold', labelsize='large',
       titleweight='bold', titlesize=15, titlepad=10)
plt.rc('animation', html='html5')

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/simulated-roads-accident-data/synthetic_road_accidents_10k.csv
/kaggle/input/simulated-roads-accident-data/synthetic_road_accidents_2k.csv
/kaggle/input/simulated-roads-accident-data/synthetic_road_accidents_100k.csv
/kaggle/input/playground-series-s5e10/sample_submission.csv
/kaggle/input/playground-series-s5e10/train.csv
/kaggle/input/playground-series-s5e10/test.csv


In [2]:
train = pd.read_csv('/kaggle/input/playground-series-s5e10/train.csv', index_col='id')
test = pd.read_csv('/kaggle/input/playground-series-s5e10/test.csv', index_col='id')

In [3]:
org = []
for n in [2, 10, 100]:
    df = pd.read_csv(f'/kaggle/input/simulated-roads-accident-data/synthetic_road_accidents_{n}k.csv')
    org.append(df)

org = pd.concat(org, axis=0)
org.head()

,road_type,num_lanes,curvature,speed_limit,lighting,weather,road_signs_present,public_road,time_of_day,holiday,school_season,num_reported_accidents,accident_risk
0,rural,2,0.72,60,daylight,clear,True,False,afternoon,False,False,2,0.37
1,highway,4,0.95,45,daylight,foggy,False,True,evening,False,True,1,0.40
2,rural,1,0.72,25,night,rainy,False,False,evening,True,False,1,0.55
3,rural,4,0.86,70,dim,foggy,True,False,morning,True,True,1,0.56
4,highway,1,0.00,60,night,rainy,True,True,morning,True,True,3,0.54


In [4]:
train = pd.concat([train, org], axis=0, ignore_index=True)

In [5]:
train = train.drop_duplicates()

In [6]:
num_cols = test.select_dtypes(include=['float64', 'int64']).columns.to_list()
cat_cols = test.select_dtypes(include='object').columns.to_list()

In [7]:
for col in num_cols:
    if test[col].dtype == 'float64':
        train[col] = train[col].astype('float32')
        test[col] = test[col].astype('float32')
        
    else:
        train[col] = train[col].astype('int32')
        test[col] = test[col].astype('int32')

for col in cat_cols:
    train[col] = train[col].astype('category')
    test[col] = test[col].astype('category')

In [8]:
features = ['road_type', 'num_lanes', 'curv_bin', 'speed_limit', 'lighting',
       'weather', 'road_signs_present', 'public_road', 'time_of_day',
       'holiday', 'school_season', 'num_reported_accidents']

target = 'accident_risk'

# Feature Engineering

In [9]:
num_cols = test.select_dtypes(include=['float32', 'int32']).columns.to_list()
cat_cols = test.select_dtypes(include='category').columns.to_list()

## Creating base risk feature based on domain knowledge

In [10]:
def risk(df):
    base_risk = (0.4 * df['curvature'] +
                 0.2 * (df['lighting'] == 'night').astype(int) +
                 0.1 * (df["weather"] != "clear").astype(int) +
                 0.2 * (df["speed_limit"] >= 60).astype(int) +
                 0.1 * (np.array(df["num_reported_accidents"] > 4).astype(int)
    ))
                 
    noise = np.random.normal(0, 0.05, df.shape[0])
    risk_score = np.clip(base_risk + noise, 0, 1)
    df["simulated_risk"] = np.round(risk_score, 2).astype('float32')

    return df

In [11]:
train = risk(train)
test = risk(test)

In [12]:
TE = []
for c in cat_cols:
    te_map = train.groupby(c)[target].mean()
    n = f"TE_{c}"
    print(f"{n}, ",end="")
 
    train[n] = train[c].map(te_map)
    test[n] = test[c].map(te_map)

    global_mean = train[target].mean()
    train[n].fillna(global_mean, inplace=True)
    test[n].fillna(global_mean, inplace=True)
    
    TE.append(n)

TE_road_type, TE_lighting, TE_weather, TE_time_of_day, 

In [13]:
def feature_engineering(df):
    
    df['speed_visibility'] = df['speed_limit'] * df['lighting'].map({'daylight': 0.5, 'dim': 1.0, 'night': 1.5}).fillna(1.0)
    df['curv_speed'] = df['curvature'] * df['speed_limit'] 
    df['curv_sq'] = df['curvature']**2
    df['speed_sq'] = df['speed_limit']**2
    df['curv_per_lane'] = df['curvature'] / (df['num_lanes'] + 1)
    df['risk_density'] = df['curv_speed'] / (df['num_lanes'] + 1)
    df['curve_night_risk'] = df['curvature'] * (df['lighting'] == 'night').astype('int32')
    df['curv_log'] = np.log1p(df['curvature'])
    df['speed_log'] = np.log1p(df['speed_limit'])
    df['meta_curvature'] = 0.3 * df['curvature']
    df['meta_night'] = 0.2 * (df['lighting'] == 'night').astype('int32')
    df['meta_weather'] = 0.1 * (df['weather'] != 'clear').astype('int32')
    df['meta_speed'] = 0.2 * (df['speed_limit'] >= 60).astype('int32')
    df['meta_accidents'] = 0.1 * (df['num_reported_accidents'] > 2).astype('int32')

    df['tight_lane'] = (df['num_lanes'] <= 2).astype(int)
   
    return df

train = feature_engineering(train)
test = feature_engineering(test)

In [14]:
for c in cat_cols:
    train[c] = pd.Categorical(train[c]).codes
    test[c] = pd.Categorical(test[c]).codes

In [15]:
train.head()

,road_type,num_lanes,curvature,speed_limit,lighting,weather,road_signs_present,public_road,time_of_day,holiday,...,risk_density,curve_night_risk,curv_log,speed_log,meta_curvature,meta_night,meta_weather,meta_speed,meta_accidents,tight_lane
0,2,2,0.06,35,0,2,False,True,0,False,...,0.700000,0.0,0.058269,3.583519,0.018,0.0,0.1,0.0,0.0,1
1,2,4,0.99,35,0,0,True,False,1,True,...,6.930000,0.0,0.688135,3.583519,0.297,0.0,0.0,0.0,0.0,0
2,1,4,0.63,70,1,0,False,True,2,True,...,8.820000,0.0,0.488580,4.262680,0.189,0.0,0.0,0.2,0.0,0
3,0,4,0.07,35,1,2,True,True,2,False,...,0.490000,0.0,0.067659,3.583519,0.021,0.0,0.1,0.0,0.0,0
4,1,1,0.58,60,0,1,False,False,1,True,...,17.399999,0.0,0.457425,4.110874,0.174,0.0,0.1,0.2,0.0,1


In [16]:
train.info()

<class 'cudf.core.dataframe.DataFrame'>
Index: 629059 entries, 0 to 629753
Data columns (total 33 columns):
 #   Column                  Non-Null Count   Dtype
---  ------                  --------------   -----
 0   road_type               629059 non-null  int8
 1   num_lanes               629059 non-null  int32
 2   curvature               629059 non-null  float32
 3   speed_limit             629059 non-null  int32
 4   lighting                629059 non-null  int8
 5   weather                 629059 non-null  int8
 6   road_signs_present      629059 non-null  bool
 7   public_road             629059 non-null  bool
 8   time_of_day             629059 non-null  int8
 9   holiday                 629059 non-null  bool
 10  school_season           629059 non-null  bool
 11  num_reported_accidents  629059 non-null  int32
 12  accident_risk           629059 non-null  float64
 13  simulated_risk          629059 non-null  float32
 14  TE_road_type            629059 non-null  float64
 15  TE_

# Splitting the data into training and testing sets

In [17]:
X = train.copy()
y = X.pop('accident_risk')
X_test = test.copy()

# Conducting optuna study


In [18]:
kf = KFold(n_splits=5, shuffle=True, random_state=2)
n_folds = 5

In [19]:
# Parameters acquired from Optuna study

xgb_param = {
    'n_estimators': 1100,
    'learning_rate': 0.05036086563157658,
    'max_depth': 5,
    'reg_alpha': 1.7078790750979551, 
    'reg_lambda': 0.04303317633957419,
    'subsample': 0.6010146163838317,
    'colsample_bytree': 0.8088494830103302,
    'eval_metric': 'rmse',
    'random_state':2, 
    'early_stopping_rounds': 100,
    'device': 'cuda',
    'objective': 'reg:squarederror',
}

cat_param = {
    'iterations': 1000,
    'learning_rate': 0.08462849871988642,
    'depth': 7,
    'l2_leaf_reg': 2.1397487441799643,
    "loss_function": "RMSE",
    'eval_metric': 'RMSE',
    'task_type':'GPU', 
    'random_state': 2,
    'early_stopping_rounds': 100,
    'verbose': 0,
}

lgbm_param = {
    "objective": "regression",
    "metric": "rmse",
    "learning_rate": 0.015047232662609947,
    "num_leaves": 124,
    "bagging_fraction": 0.6338824787124612,
    "reg_alpha": 0.2869136891325499,
    "reg_lambda": 2.4855353033908525,
    "max_depth": 9,
    "n_estimators": 908,
    'min_data_in_leaf': 70,
    "device_type": "gpu",
    "random_state": 2,
    'verbose': -1,
    'n_jobs': -1,
}

xgb = XGBRegressor(**xgb_param)
cat = CatBoostRegressor(**cat_param)
lgbm = LGBMRegressor(**lgbm_param)

In [20]:
meta_train = np.zeros((len(y), 3))
meta_test = np.zeros((len(X_test), 3))

In [21]:
for fold, (train_index, valid_index) in enumerate(kf.split(X, y), start=1):
    X_train, X_valid = X.iloc[train_index], X.iloc[valid_index]
    y_train, y_valid = y.iloc[train_index], y.iloc[valid_index]

    xgb.fit(X_train, y_train, eval_set=[(X_valid, y_valid)], verbose=0)
    cat.fit(X_train, y_train, eval_set=[(X_valid, y_valid)])
    lgbm.fit(X_train, y_train, eval_set=[(X_valid, y_valid)], callbacks=[early_stopping(100), log_evaluation(period=0)])
        
    meta_train[valid_index, 0] = xgb.predict(X_valid)
    meta_train[valid_index, 1] = cat.predict(X_valid)
    meta_train[valid_index, 2] = lgbm.predict(X_valid)

    meta_test[:, 0] += xgb.predict(X_test) / n_folds
    meta_test[:, 1] += cat.predict(X_test) / n_folds
    meta_test[:, 2] += lgbm.predict(X_test) / n_folds

meta_model = BayesianRidge(n_iter=2000).fit(meta_train, y)
test_preds = meta_model.predict(meta_test)    

1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.


Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[908]	valid_0's rmse: 0.056177


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:160: UserWarning: [07:49:57] WARNING: /workspace/src/common/error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  warnings.warn(smsg, UserWarning)


Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[908]	valid_0's rmse: 0.0559871
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[907]	valid_0's rmse: 0.0559819
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[908]	valid_0's rmse: 0.0563029
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[908]	valid_0's rmse: 0.0562506


In [22]:
oof_predictions = meta_model.predict(meta_train)

# Calculate overall RMSE
stacking_rmse = np.sqrt(mean_squared_error(y, oof_predictions))
print(f"\n{'='*50}")
print(f"Stacking Model OOF RMSE: {stacking_rmse:.5f}")
print(f"{'='*50}")


Stacking Model OOF RMSE: 0.05599


In [23]:
sub = pd.read_csv('/kaggle/input/playground-series-s5e10/sample_submission.csv')
sub['accident_risk'] = test_preds
sub.to_csv('submission.csv', index=False)
sub.head()

,id,accident_risk
0,517754,0.295464
1,517755,0.119579
2,517756,0.193463
3,517757,0.341217
4,517758,0.381808
